In [1]:
import pandas as pd
import os
import numpy as np
import requests
import matplotlib.pyplot as plt
import time
import seaborn as sns
import datetime
from datetime import date

# The first step is to load my CSV files from their folders. The below code will load each file from a given folder. I have separated my folders, I will run this code twice